# Memoria
## Grupo 10
### Integrantes
- Xiao Peng Ye
- Zhengyu Ye
- Juan Diego Valencia Marin

## Índice
1. Introducción
2. Algoritmo implementado
3. Funcionamiento específico
4. Comparación con otro algoritmo
5. Conclusiones
6. Bibliografía

## Introducción
Esta práctica se basa en la implementación de un algoritmo evolutivo y de su comparación con otro de una biblioteca estudiada en esta asignatura, en concreto se realizará el algoritmo de [**Evolución Diferencial (DE)**](https://es.wikipedia.org/wiki/Evoluci%C3%B3n_diferencial) y la biblioteca [**SciPy**](https://docs.scipy.org/doc/scipy/reference/) donde encontraremos este mismo algoritmo de evolución diferencial.

El algoritmo Evolución Diferencial es un método de optimización que está dentro de la computación evolutiva (rama de la inteligencia artificial inspirada en los mecanismos de la evolución biológica).

La biblioteca SciPy, que viene de las siglas Scientific Python, es una biblioteca de cálculo científico que proporciona funciones de utilidad para optimización, estadísticas y procesamiento de señales.


## Algoritmo implementado
El algoritmo de **Evolución Diferencial (DE)** ha sido el realizado en esta práctica, este algoritmo tiene una configuración específica, la *mutación se utiliza la estrategia (de/rand/1)*, *selección de forma uniforme*, *cruzamiento binomial* y un *reemplazo elitista y posición a posición*.

La idea del DE es generar una población de nuevas soluciones a partir de perturbar soluciones pertenecientes a la población de ese momento.

El funcionamiento de este algoritmo es el siguiente:

1. Crear una población inicial.
2. Determinar un individuo de la población en función de la estrategia de selección.
3. Generar nuevos individuos según el operador de mutación.
4. Aplicar el cruzamiento/recombinación.
5. Reemplazar el individuo seleccionado.
6. Volver al paso 2 hasta obtener la mejor solución.

## Funcionamiento específico
- **Uniform Selection.**

    Se escogen tres vectores diferentes de forma aleatoria dentro de la población obtenida, con los que trabajaremos junto al **Target vector** del que se hablará más adelante. A estos tres vectores se les llama **Donor vectors**.
    
	
- **Mutation (de/rand/1).**
	
    Se utiliza la estrategia de/rand/1, donde se escogen tres vectores diferentes de forma aleatoria. A continuación se realiza la siguiente operación:
    
    V<sub>i,g</sub> = x<sub>r<sub>0</sub>,g</sub> + F(x<sub>r<sub>1</sub>,g</sub> − x<sub>r<sub>2</sub>,g</sub>)
 
 Donde:
 - F es una constante entre 0 y 2, 1 en nuestro caso.
 - V<sub>i,g</sub> es el resultado de la mutación.


- **Binomial Crossover.**

  En el cruzamiento binomial, en donde aparece el **trial vector (u<sub>i,g</sub>)**, creado por los elementos del target vector (x<sub>i,g</sub>) y del donor vector (v<sub>i,g</sub>), además de una probabilidad **CR**, 0.1 en nuestro caso.
  
  u<sub>i,g</sub> =
  
  v<sub>i,g</sub> si rand <= CR
         o
  x<sub>i,g</sub> si rand > CR
  
  Donde:
  - *rand* es un número aleatorio comprendido entre 0 y 1.
  - *CR* es la probabilidad antes descrita.


- **Elitist Replacement.**

	Este reemplazo es el más habitual, en el que se compara el *target vector* con el *trial vector* y aquel con la función de menor valor es el admitido en la siguiente generación.
    

## Comparación

En este apartado se procederá a realizar la comparación del algoritmo **DE** implementado por nosotros, con otras implementaciones provenientes de una biblioteca. Se incluirán los resultados de ambas implementaciones representadas mediante tablas y algunos comentarios explicativos.

En los siguientes fragmentos, se importan las librerías y las funciones a utilizar, junto con la creación de ambos algoritmos.
En particular, se utilizan la optimización de **SciPy** y **Pyade (SADE en particular)** que proporcionan funciones para maximizar o minimizar funciones objetivas. Y algunas de las [funciones de optimización](https://en.wikipedia.org/wiki/Test_functions_for_optimization) como la *sphere, ackley, rosenbrock,...* entre otras, dentro de la librería [Benchmarks](https://deap.readthedocs.io/en/master/api/benchmarks.html#deap.benchmarks.sphere). 

In [1]:
from scipy.optimize import differential_evolution
from group10.EA import EA
from benchmarks.functions import sphere, ackley, rosenbrock, rastrigin, griewank, schwefel_2_21,schwefel_2_22, schwefel_1_2, extended_f_10, bohachevsky, schaffer
import pandas as pd
import numpy as np
import pyade.sade as sade

In [2]:
def DE_scipy(m_function,bounds,p_size,iteration):
    result = differential_evolution(m_function, bounds, popsize=p_size, polish=False, maxiter=iteration,strategy="rand1bin")
    return result

In [3]:
def DE_propio(m_function,bounds,p_size,iteration):
    miEA = EA(m_function, bounds, p_size)
    miEA.run(iteration)
    bestGenome = miEA.best()
    return bestGenome

In [4]:
def DE_SADE(m_function,bounds,probsize,p_size,iteration):
    params = sade.get_default_params(dim=probsize)
    params['bounds'         ] = np.array([[bounds[0], bounds[1]]] * probsize)
    params['max_evals'      ] = p_size*iteration
    params['population_size'] = p_size
    params['func'           ] = m_function
    results = []
    # for i in range(iteration):
    _, fitness = sade.apply(**params)
    results.append(fitness)

    return results

En esta parte, se procede a utilizar todas las funciones de optimización dentro de la librería benchmarks, junto con el algoritmo DE implementado por nuestra parte y las librerías SciPy y SADE.

In [5]:
benchmark=[sphere, ackley, rosenbrock, rastrigin, griewank, schwefel_2_21,
             schwefel_2_22, schwefel_1_2, extended_f_10, bohachevsky, schaffer]

In [6]:
resultMI={}

for func in benchmark:
    resultMI[func.__name__] = DE_propio(func, [(-100, 100)] * 10,50,100).solution

In [7]:
resultDE={}
for func in benchmark:
    resultDE[func.__name__] = DE_scipy(func, [(-100, 100)] * 10,50,1).x

In [8]:
resultSADE={}
for func in benchmark:
    resultSADE[func.__name__] = DE_SADE(func, (-100, 100), 10,50,1)

Ahora se procede a obtener los resultados y almacenarlos en un par de tablas para falicitar su visualización y comprensión. Se obtendrán algunos valores estadísticos como la media, desviación, mínimo, máximo y mediana.

In [9]:
ind=['sphere', 'ackley', 'rosenbrock', 'rastrigin', 'griewank', 'schwefel_2_21',
     'schwefel_2_22', 'schwefel_1_2', 'extended_f_10', 'bohachevsky', 'schaffer']
col=['avg','std','min','max','med']

In [10]:
df_b=pd.DataFrame(index=ind,columns=col)

In [11]:
df_m=pd.DataFrame(index=ind,columns=col)

In [12]:
df_d=pd.DataFrame(index=ind,columns=col)

Una vez creadas las tablas, se procede a guardar los datos para poder visualizarlos.

In [13]:
algNames = ["MI", "DE","SADE"]

results_avg = {}
results_std = {}
results_min = {}
results_max = {}
results_median = {}


for n in algNames:
    results_avg[n] = []
    results_std[n] = []
    results_min[n] = []
    results_max[n] = []
    results_median[n] = []


for func in benchmark:
    f = func.__name__
    results_avg["MI"].append(np.mean(resultMI[f]))
    results_avg["DE"].append(np.mean(resultDE[f]))
    results_avg["SADE"].append(np.mean(resultSADE[f]))
    results_std["MI"].append(np.std(resultMI[f]))
    results_std["DE"].append(np.std(resultDE[f]))
    results_std["SADE"].append(np.std(resultSADE[f]))
    results_min["MI"].append(np.min(resultMI[f]))
    results_min["DE"].append(np.min(resultDE[f]))
    results_min["SADE"].append(np.min(resultSADE[f]))
    results_max["MI"].append(np.max(resultMI[f]))
    results_max["DE"].append(np.max(resultDE[f]))
    results_max["SADE"].append(np.max(resultSADE[f]))
    results_median["MI"].append(np.median(resultMI[f]))
    results_median["DE"].append(np.median(resultDE[f]))
    results_median["SADE"].append(np.median(resultSADE[f]))

In [14]:
df_b.loc[:,'avg']=results_avg["MI"]
df_m.loc[:,'avg']=results_avg["DE"]
df_d.loc[:,'avg']=results_avg["SADE"]
df_b.loc[:,'std']=results_std["MI"]
df_m.loc[:,'std']=results_std["DE"]
df_d.loc[:,'std']=results_std["SADE"]
df_b.loc[:,'max']=results_max["MI"]
df_m.loc[:,'max']=results_max["DE"]
df_d.loc[:,'max']=results_max["SADE"]
df_b.loc[:,'min']=results_min["MI"]
df_m.loc[:,'min']=results_min["DE"]
df_d.loc[:,'min']=results_min["SADE"]
df_b.loc[:,'med']=results_median["MI"]
df_m.loc[:,'med']=results_median["DE"]
df_d.loc[:,'med']=results_median["SADE"]

### Visualización de las tablas

En este punto, se visualizan los datos de todas las implementaciones mediante tablas:

- **Tabla de DE de la biblioteca SciPy:**

In [15]:
df_b

,avg,std,min,max,med
sphere,0.654609,2.530447,-3.992671,4.430975,1.399140
ackley,18.400000,54.116910,-60.000000,100.000000,12.000000
rosenbrock,2.290646,7.884021,-5.538144,24.471041,1.395956
rastrigin,-0.792375,2.903469,-4.928530,4.784870,-0.640684
griewank,-0.184679,2.481950,-4.771186,3.184851,-0.081057
schwefel_2_21,-4.434521,10.337093,-16.106182,15.234144,-8.562615
schwefel_2_22,0.227617,4.442384,-11.019528,5.935192,-0.103948
schwefel_1_2,-4.242250,19.506662,-42.420516,21.331215,-10.629697
extended_f_10,-0.930450,7.384353,-16.904915,11.216873,0.691503
bohachevsky,0.412739,2.940650,-3.742725,6.738843,0.480011


- **Tabla de DE de la biblioteca SADE:**

In [16]:
df_b

,avg,std,min,max,med
sphere,0.654609,2.530447,-3.992671,4.430975,1.399140
ackley,18.400000,54.116910,-60.000000,100.000000,12.000000
rosenbrock,2.290646,7.884021,-5.538144,24.471041,1.395956
rastrigin,-0.792375,2.903469,-4.928530,4.784870,-0.640684
griewank,-0.184679,2.481950,-4.771186,3.184851,-0.081057
schwefel_2_21,-4.434521,10.337093,-16.106182,15.234144,-8.562615
schwefel_2_22,0.227617,4.442384,-11.019528,5.935192,-0.103948
schwefel_1_2,-4.242250,19.506662,-42.420516,21.331215,-10.629697
extended_f_10,-0.930450,7.384353,-16.904915,11.216873,0.691503
bohachevsky,0.412739,2.940650,-3.742725,6.738843,0.480011


- **Tabla de DE con nuestra implementación:**

In [17]:
df_m


,avg,std,min,max,med
sphere,7.649395,21.926159,-25.385472,58.043125,3.620130
ackley,-15.433013,51.490269,-79.816462,66.988401,-39.119388
rosenbrock,5.199235,25.183851,-49.271378,44.666327,8.727724
rastrigin,20.454436,19.446675,-7.194109,54.539926,24.287943
griewank,0.345059,21.810225,-26.401906,42.124134,-0.107374
schwefel_2_21,12.151889,24.228086,-34.822227,44.843282,21.466099
schwefel_2_22,9.286793,46.908649,-93.698648,92.284188,6.003610
schwefel_1_2,0.677340,33.533422,-56.214217,72.410290,-9.230574
extended_f_10,-18.118704,34.080112,-65.767190,63.130915,-22.885302
bohachevsky,-3.380234,29.973498,-80.693352,34.978313,-0.005558


Ya con todos los resultados de los algoritmos, se comprobarán los algoritmos mediante la prueba de **Kruskal-Wallis**. Esta prueba es una comprobación más en profundidad, pues se comprueba función a función y se pueden observar ciertas diferencias dependiendo de la implementación.

In [18]:
from scipy.stats import kruskal

for func in benchmark:
    f = func.__name__
    res = kruskal(resultMI[f], resultDE[f], resultSADE[f])
    print("Results for the "+f+" function: "+str(res.pvalue))


Results for the sphere function: 0.610481985585928
Results for the ackley function: 0.29574992126496136
Results for the rosenbrock function: 0.19723159662080514
Results for the rastrigin function: 0.06969432648219902
Results for the griewank function: 0.9444591369948634
Results for the schwefel_2_21 function: 0.26485535596213583
Results for the schwefel_2_22 function: 0.29692202572789594
Results for the schwefel_1_2 function: 0.8337529180751763
Results for the extended_f_10 function: 0.06216755258803258
Results for the bohachevsky function: 0.5043849138615993
Results for the schaffer function: 0.7322097138745831


De forma semejante a la anterior, se realizará la prueba de **Friedman**, que es una prueba no paramétrica. Aunque esta no es tan específica, pues se realiza considerando las medias, y no las funciones como la de Kruskal-Wallis.

In [19]:
from scipy.stats import friedmanchisquare

friedmanchisquare(results_avg["MI"], results_avg["DE"], results_avg["SADE"])



FriedmanchisquareResult(statistic=7.818181818181813, pvalue=0.020058727982721684)

Por último, utilizaremos la librería [scikit-posthocs](https://scikit-posthocs.readthedocs.io/en/latest/) para realizar una comparación directa de las tres posibles implementaciones. Esta comparación se realizará mediante el método de comparación por pares, en donde se comparan los algoritmos dos a dos obteniendo unos resultados más específicos.

En la tabla se puede observar que los resultados son mejores cuando el valor se acerca más a 0, siendo la comparación de un algoritmo consigo mismo 1. Por lo que podemos afirmar que nuestra implementación no es tan eficiente como las demás porque los valores son ligeramente mayores a los de las librerías. Y con este mismo razonamiento, el algoritmo SADE es un poco mejor que el de DE.

In [20]:
import scikit_posthocs as sp

data = pd.DataFrame({"algs": ["MI" ]*len(results_avg["MI"]) +
                             ["DE"]*len(results_avg["DE"]) +
                            ["SADE"]*len(results_avg["SADE"]),
                     "vals": results_avg["MI"] +
                             results_avg["DE"] +
                            results_avg["SADE"]})

sp.posthoc_wilcoxon(data, val_col='vals', group_col='algs', p_adjust = 'holm')




,MI,DE,SADE
MI,1.000000,0.413086,0.096680
DE,0.413086,1.000000,0.412109
SADE,0.096680,0.412109,1.000000


## Conclusiones
Este trabajo se ha enfocado en la comparación de los resultados del algoritmo de **Evolución Diferencial (DE)** implementado por nosotros con unas estrategias y  métodos específicos, con el mismo algoritmo (DE) proveniente de la librería SciPy. En los resultados, observamos... (poner algo)

En cuanto al algoritmo DE, vemos su utilidad y nos damos cuenta de que los resultados son bastante buenos, aunque no los utilicemos de una forma óptima. Además, como ya veníamos de realizar el algoritmo genético (GA) del anterior trabajo, vemos que hay bastantes algoritmos dentro de la computación evolutiva y nos percatamos de la gran utilidad de ellos.

(Pasando a un enfoque más personal sobre este trabajo) (cambiar) sobre este trabajo, encontramos interesante la posibilidad de comparar estos algoritmos, de este modo hemos podido adentrarnos e indagar más en profundidad sobre ellos, conociendo así las soluciones óptimas y las que no son óptimas. 

## Bibliografía
https://es.wikipedia.org/wiki/Evoluci%C3%B3n_diferencial

https://docs.scipy.org/doc/scipy/reference/

https://docs.scipy.org/doc/scipy/reference/optimize.html

https://en.wikipedia.org/wiki/Differential_evolution

https://www.sciencedirect.com/science/article/pii/S2215098618323401#e0025

https://www.sciencedirect.com/science/article/abs/pii/S1568494609000325

https://www.hindawi.com/journals/jcse/2013/462706/